# Vulnerability Assessment Pilot
This notebook demonstrates on-going development of climate adaptation vulnerability assessment (CAVA) support using climate data in the Analytics Engine. 

To execute a given 'cell' of this notebook, place the cursor in the cell and press the 'play' icon, or simply press shift+enter together. Some cells will take longer to run, and you will see a [$\ast$] to the left of the cell while AE is still working.

**Intended Application**: As a user, I want to **<span style="color:#FF0000">access climate projections data for my vulnerability assessment report</span>** by:
1. Retrieve data metrics required for planning needs

**Runtime**: With the default settings, this notebook takes approximately **less than 10 minutes** to run from start to finish. Modifications to selections may increase the runtime. <br>*This notebook is currently in progress, runtime will change as improvements and further analyses are added.*

### Step 0: Set-up

First, we'll import the Python library [climakitae](https://github.com/cal-adapt/climakitae), our AE toolkit for climate data analysis, along with this specific functions from that library that we'll use in this notebook, as well as any other necessary Python libraries to aid in analysis.

In [ ]:
import climakitae as ck
import pandas as pd

from climakitae.explore.vulnerability import cava_data

### Step 1: Import locations
Now we'll read in point-based locations that we want to retrieve data for. For custom inputs, there are two options: (1) Input a single pair of latitude - longitude values; and (2) Import a csv file of locations that will each run. In the code below we show what each option looks like. 

Functionality to assess over a gridded area (region) is in the works as well!

In [ ]:
## select a single custom location
# your_lat = LAT
# your_lon = LON

To import your own custom locations, we recommend putting your csv file in the same folder as this notebook for ease:
1. Drag and drop a csv file into the file tree on the left hand side; or
2. Use the `upload` button (the "up arrow" symbol next to the large blue plus symbol above the file tree). 

<span style="color:#FF0000">**Formatting note**</span>: For the code cells below to work, there must be **2 columns labeled `lat` and `lon`**. Functionality to accept different labeling is forthcoming!

In the cell below, we read the csv file in. We extracted three random locations from the HadISD station list as an example here -- you'll want to replace with your own locations file!

In [ ]:
# Read in dummy locations from `stations_csv` file
from climakitae.core.paths import stations_csv_path
from climakitae.util.utils import read_csv_file
example_locs = read_csv_file(stations_csv_path, index_col=0)[['LAT_Y', 'LON_X']].rename(columns={'LAT_Y': 'lat', 'LON_X': 'lon'})

In [ ]:
# select a location from the location list
one_loc = example_locs.loc[example_locs.index == 0]
loc_lat = one_loc.lat.values[0]
loc_lon = one_loc.lon.values[0]
print(loc_lat, loc_lon)

### Step 2: Retrieve metric data

The `cava_data` funciontality is designed to provide flexibility over customizable metric calculation. Below is a table outlining all avaialble arguments to the `cava_data` function. The "Required" flag notes whether the argument must be passed to start generating data. Input options for each argument is provided, as well as whether a setting is required for any of the required selections. 

| Argument | Required? | Options | Argument required for | Notes |
|----------|-----------|---------|-----------------------|-------|
|location | Yes | Pass a location via csv. | |*In progress direct input*|
|variable | Yes | "Air Temperature at 2m", "NOAA Heat Index"| |*In progress "Precipitation (total)"* |
|approach | Yes | "time", "gwl" | | *In progress GWL*|
|downscaling_method | Yes | "dynamical", "statistical"| | *In progress statistical*|
|time_start_year | No | Numerical (min is 1981) | Required for **approach=time** |
|time_end_year | No | Numerical (max is 2100) | Required for **approach=time**|
|historical_data| No | "Historical Climate", "Historical Reconstruction" | | *In progress Historical Reconstruction*|
|ssp_data | No | "[SSP2-4.5]", "[SSP3-7.0]", "[SSP5-8.5]" | Required for **approach=time** |Dynamical only has SSP3-7.0, Statisical has all 3|
|warming_level| No| "1.5", "2.0", "3.0" | Required for **approach=gwl**|
|metric_calc| No | "max", "min" |
|heat_idx_threshold | No | Numerical | Required for **variable="NOAA Heat Index"** |
|one_in_x | No | Numerical |
|percentile | No | Numerical (0-100) | | Default set to 50th percentile |
|season| No | "summer", "winter", "all" |
|units | Optional | Temp/Heat Index: "degF", "degC", "K". Precip: "mm", "inches" | | Default for temp/heat index is DegF. *In progress Precip*|
|wrf_bc| Optional | True, False | | Only applicalble for **downscaling_method="dynamical"**. Default is True |
|export_method| Optional | "off-ramp", "calculate", "both" |  |Default set to "both" |
|file_format | Optional | "NetCDF", "csv" |  |Default set to "NetCDF" |
|separate_files | Optional | True, False |

For example, if I wanted to calculate **Likely summer day high temperature for 2030, where likely is the 75th percentile, in Celsius, using WRF data, and export only the calculated metric data** I would input:<br>
data = cava_data(<br>
   location = example_locs, <br>
   variable = "Air Temperature at 2m", <br>
   approach = "time", <br>
   downscaling_method = "Dynamical", <br>
   time_start_year = 2030, <br>
   time_start_end = 2050, <br>
   ssp_data = ["SSP3-7.0"], <br>
   metric_calc = "max", <br>
   percentile = 75, <br>
   season = "summer", <br>
   units = "degC", <br>
   export_method = "calculate"<br>
)

In [ ]:
data = cava_data(
    example_locs.iloc[:1],
    time_start_year=2030,
    time_end_year=2050,
    units="degF",
    downscaling_method="Dynamical",  # default for now ## mandatory
    approach="time",  
    warming_level='3.0',
    wrf_bc=True,
    historical_data="Historical Climate",  # or "historical reconstruction"
    ssp_data=["SSP3-7.0"],
    variable="Air Temperature at 2m",  ## mandatory, must eventually accept temp, precip, or heat index
    metric_calc="max", 
    heat_idx_threshold=None, # Heat Index Threshold
    one_in_x=2, # One-in-X
    percentile=None, # Likeliness
    season="summer",
    export_method="calculate",  # off-ramp, full calculate, both
    separate_files=True, # Toggle to determine whether or not the user wants to separate climate variable information into separate files
    file_format="NetCDF",
)

---

### Appendix: Table Generation Sample Code

Below, you'll find code that generates a table with different climate data metrics used in a CAVA Report. Feel free to run it and check it out! It is still very much in progress.

In [ ]:
from climakitae.explore.vulnerability_table import create_vul_table

In [ ]:
df = create_vul_table(example_locs.iloc[10])